Installing the Julep Client

In [66]:
!pip install julep

In [67]:
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4()

### Create Julep Client


In [68]:
from julep import Client

JULEP_API_KEY = "YOUR_JULEP_API_KEY"

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="production")

### Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the Agent section in [Julep Concepts](https://docs.julep.ai/docs/concepts/agents).

In [69]:
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Julep Email Assistant",
    about=(
        "You are an agent that handles emails for julep users."
        + " Julep is a platform for creating kick-ass AI agents."
    ),
    model="gpt-4o-mini",
)

### Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tasks).

In [76]:
import yaml

MAILGUN_PASSWORD = "YOUR_MAILGUN_PASSWORD"

task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/src/schemas/create_task_request.json
name: Julep Email Assistant
description: A Julep agent that can send emails and search the documentation.

########################################################
####################### INPUT SCHEMA ###################
########################################################
input_schema:
  type: object
  properties:
    from:
      type: string
    to:
      type: string
    subject:
      type: string
    body:
      type: string

                          
########################################################
####################### TOOLS ##########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: send_email
  type: integration
  integration:
    provider: email
    setup:
      host: smtp.mailgun.org
      password: {MAILGUN_PASSWORD}
      port: 587
      user: postmaster@email.julep.ai

- name: search_docs
  type: system
  system:
    resource: agent
    subresource: doc
    operation: search
  
########################################################
####################### MAIN WORKFLOW ##################
########################################################

main:
# Step 0: Prompt the user for the email details
- prompt: |-
    $ f'''You are {{ agent.name }}. {{ agent.about }}

    A user with email address {{ _['from'] }} has sent the following inquiry:
    ------
      Subject: {{ _.subject }}

      {{ _.body }}
    ------

    Can you generate a query to search the documentation based on this email?
    Just respond with the query as is and nothing else.'''

  unwrap: true

# Step 1: Search the documentation
- tool: search_docs
  arguments:
    agent_id: {agent.id}
    text: $ _
    
- prompt: >-
    $ f'''You are {{ agent.name }}. {{ agent.about }}

    A user with email address {{ steps[0].input['from'] }} has sent the following inquiry:
    ------
      Subject: {{ steps[0].input.subject }}

      {{ steps[0].input.body }}
    ------

    Here are some possibly relevant snippets from the julep documentation:
    {{ '\\n'.join([snippet.content for doc in _.docs for snippet in doc.snippets]) }}
    
    ========

    Based on the above info, craft an email body to respond with as a json object.
    The json object must have `subject` and `body` fields.'''
  response_format:
    type: json_object
    
  unwrap: true

# Step 3: Extract the email
- evaluate:
    subject: $ extract_json(_)['subject']
    body: $ extract_json(_)['body']

# Step 4: Send the email
- tool: send_email
  arguments:
    body: $ _.body
    from: postmaster@email.julep.ai
    subject: $  _.subject
    to: $ steps[0].input['from']
""")

In [77]:
task = client.tasks.create_or_update(
    agent_id=AGENT_UUID,
    task_id=TASK_UUID,
    **task_def,
)

### Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

To learn more about executions, please refer to the `Executions` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/execution).

In [84]:
execution = client.executions.create(
    task_id=task.id,
    input={"from": "diwank@julep.ai", "to": "help@agents.new", "subject": "what's up", "body": "sup"},
)

In [85]:
client.executions.get(execution.id)

Execution(id='067fd065-3722-7af4-8000-817a972d8c7f', created_at=datetime.datetime(2025, 4, 14, 12, 57, 55, 446101, tzinfo=datetime.timezone.utc), input={'to': 'help@agents.new', 'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up"}, status='running', task_id='fdab8052-df0d-4f36-9c5e-01dcd5e0ba3c', updated_at=datetime.datetime(2025, 4, 14, 12, 57, 56, 742329, tzinfo=datetime.timezone.utc), error=None, metadata={}, output={'docs': [], 'time': 0.01}, transition_count=3)

### Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [87]:
execution_transitions = client.executions.transitions.list(
    execution_id=execution.id).items

for transition in reversed(execution_transitions):
    print("Type: ", transition.type)
    print("output: ", transition.output)
    print("-" * 100)

Type:  init
output:  {'to': 'help@agents.new', 'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up"}
----------------------------------------------------------------------------------------------------
Type:  step
output:  "what's up" OR "sup" OR "general inquiry"
----------------------------------------------------------------------------------------------------
Type:  step
output:  {'docs': [], 'time': 0.01}
----------------------------------------------------------------------------------------------------
Type:  step
output:  ```json
{
  "subject": "Hello from Julep!",
  "body": "Hi there! Thanks for reaching out. We're here to help you create amazing AI agents. If you have any specific questions or need assistance with the platform, feel free to ask. Looking forward to hearing from you!"
}
```
----------------------------------------------------------------------------------------------------
Type:  step
output:  {'body': "Hi there! Thanks for reaching out. We're here 

## Related Concepts

- [Agents](https://docs.julep.ai/concepts/agents)
- [Tasks](https://docs.julep.ai/concepts/tasks)
- [Tasks](https://docs.julep.ai/concepts/tasks)
